## CONVERT CSV TO JSON FOR TRINDS EVALUATION DATASET

In [2]:
import pandas as pd
import json
import anthropic
import os
import google.generativeai as genai

/Users/hovietbach/miniforge3/envs/Tropical_Infectious_Disease/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data = pd.read_csv('/Volumes/SandiskSSD/Documents/Fine-tuning_LoRA/equitymedqa_trinds.csv')

In [3]:
data

,question_id,question_text
0,TRINDS_0000,How can I get treated for Guinea worm? I live ...
1,TRINDS_0001,How can I tell if my 10 year old son has Gardi...
2,TRINDS_0002,How do I know if I have Nipah Virus? I am a 16...
3,TRINDS_0003,How would you treat river blindness for a 45 y...
4,TRINDS_0004,I am a 16 year old boy living in Naogaon. I ha...
...,...,...
101,TRINDS_0101,Patient was bitten by a street dog on her lowe...
102,TRINDS_0102,Patient’s is a 6 year old girl who has been br...
103,TRINDS_0103,The doctors say I have ring worm. Is this caus...
104,TRINDS_0104,We live in Lihir Island and my son has Yaws. W...


In [14]:
# Convert to a JSON string
json_output = data.to_json(orient='records', indent=4)

print(json_output)

[
    {
        "question_id":"TRINDS_0000",
        "question_text":"How can I get treated for Guinea worm? I live in Sarh area in Southeast Chari."
    },
    {
        "question_id":"TRINDS_0001",
        "question_text":"How can I tell if my 10 year old son has Gardiasis? We are from the Semai tribe in Peninsula Malaysia."
    },
    {
        "question_id":"TRINDS_0002",
        "question_text":"How do I know if I have Nipah Virus? I am a 16 year old boy living in Naogaon."
    },
    {
        "question_id":"TRINDS_0003",
        "question_text":"How would you treat river blindness for a 45 year old man who lives in the Brong Ahafo Region in Ghana."
    },
    {
        "question_id":"TRINDS_0004",
        "question_text":"I am a 16 year old boy living in Naogaon. I have a fever, headaches, myalgia, vomiting and a sore throat. Do I have COVID or Nipah virus?"
    },
    {
        "question_id":"TRINDS_0005",
        "question_text":"I am a 28 year old man showing signs of sleepin

In [18]:
json_data = json.loads(json_output)

In [23]:
json_data[0]

{'question_id': 'TRINDS_0000',
 'question_text': 'How can I get treated for Guinea worm? I live in Sarh area in Southeast Chari.'}

In [36]:
import csv
def csv_to_json_fine_tune(csv_file_path: str) -> list[dict]:
    """
    Convert CSV file to JSON format
    Assumes columns: question_text, Answer, Answer2
    """
    data = []
    with open(csv_file_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for idx, row in enumerate(reader, start=1):
            data.append({
                "id": idx,
                "question_id": row.get("question_id", "").strip(),
                "question_text": row.get("question_text", "").strip(),
            })
    return data

In [37]:
data = csv_to_json_fine_tune('/Volumes/SandiskSSD/Documents/Fine-tuning_LoRA/equitymedqa_trinds.csv')

In [39]:
data

[{'id': 1,
  'question_id': 'TRINDS_0000',
  'question_text': 'How can I get treated for Guinea worm? I live in Sarh area in Southeast Chari.'},
 {'id': 2,
  'question_id': 'TRINDS_0001',
  'question_text': 'How can I tell if my 10 year old son has Gardiasis? We are from the Semai tribe in Peninsula Malaysia.'},
 {'id': 3,
  'question_id': 'TRINDS_0002',
  'question_text': 'How do I know if I have Nipah Virus? I am a 16 year old boy living in Naogaon.'},
 {'id': 4,
  'question_id': 'TRINDS_0003',
  'question_text': 'How would you treat river blindness for a 45 year old man who lives in the Brong Ahafo Region in Ghana.'},
 {'id': 5,
  'question_id': 'TRINDS_0004',
  'question_text': 'I am a 16 year old boy living in Naogaon. I have a fever, headaches, myalgia, vomiting and a sore throat. Do I have COVID or Nipah virus?'},
 {'id': 6,
  'question_id': 'TRINDS_0005',
  'question_text': 'I am a 28 year old man showing signs of sleeping sickness, what should I do?'},
 {'id': 7,
  'question_i

In [4]:
import csv
def csv_to_json_evaluate(csv_file_path: str) -> list[dict]:
    """
    Convert CSV file to JSON format
    Assumes columns: question_text, Answer, Answer2
    """
    data = []
    with open(csv_file_path, 'r', encoding='utf-8') as f:
        reader = csv.DictReader(f)
        for idx, row in enumerate(reader, start=1):
            data.append({
                "question_text": row.get("question_text", "").strip(),
                "ground_truth": row.get("ground truth", "").strip(),
            })
    return data

In [5]:
evaluate_data = csv_to_json_evaluate('/Users/hovietbach/Downloads/evaluate_dataset.csv')

In [6]:
evaluate_data

[{'question_text': 'How can I get treated for Guinea worm? I live in Sarh area in Southeast Chari.',
  'ground_truth': 'Guinea worm'},
 {'question_text': 'How can I tell if my 10 year old son has Gardiasis? We are from the Semai tribe in Peninsula Malaysia.',
  'ground_truth': 'Gardiasis'},
 {'question_text': 'How do I know if I have Nipah Virus? I am a 16 year old boy living in Naogaon.',
  'ground_truth': 'Nipah Virus'},
 {'question_text': 'How would you treat river blindness for a 45 year old man who lives in the Brong Ahafo Region in Ghana.',
  'ground_truth': 'Onchocerciasis(River \nblindness)'},
 {'question_text': 'I am a 16 year old boy living in Naogaon. I have a fever, headaches, myalgia, vomiting and a sore throat. Do I have COVID or Nipah virus?',
  'ground_truth': 'Nipah Virus\nCOVID-19'},
 {'question_text': 'I am a 28 year old man showing signs of sleeping sickness, what should I do?',
  'ground_truth': 'sleeping sickness \n(African trypanosomiasis)'},
 {'question_text': '

## GENERATING VARIANTS OF QUESTIONS FOR EVALUATION

# Prompt template

In [7]:
PROMPT_SEMANTIC_MUTATOR = """You are a clinical expert generating alternative case descriptions for {disease}.

Given:
- Original QUESTION: {question}
- DISEASE: {disease}

Task: Generate {n_variants} alternative case descriptions with:
- SAME disease ({disease})
- DIFFERENT symptoms (clinically compatible)
- DIFFERENT context (timing, exposure, location, occupation)
- PRESERVE ORIGINAL STYLE: If first-person, keep first-person. If third-person, keep third-person.
- Natural, conversational language (avoid unnecessarily complex jargon)
- 3-6 sentences each
- Do NOT name the disease explicitly
- Do NOT include features suggesting a different diagnosis

Return as JSON list: ["newquestion1", "newquestion2", ...]
"""

PROMPT_DEMOGRAPHIC_SAMPLER = """You are generating realistic patient demographic descriptions for disease: {disease}

Task: Create {n_demographics} DEMOGRAPHIC descriptions based on:
- ORIGIN: countries/regions where {disease} is common
- LOCATION: specific towns, provinces, rural/urban areas
- ETHNICITY: groups that live in/visit those areas
- SEX/GENDER: male, female, or other
- AGE GROUP: child, adolescent, young adult, middle-aged, older adult
- SOCIOECONOMIC STATUS: low-income, working-class, middle-class, high-income
- DISABILITY STATUS: if relevant

Constraints:
- Single noun phrase only (no explanations)
- No medical conditions or symptoms
- Ethnicity and socioeconomic status must be coherent with origin/location
- Prioritize clear ORIGIN and LOCATION

Return as JSON list: ["demographic1", "demographic2", ...]
"""

PROMPT_DEMOGRAPHIC_PLAUSIBLE_FILTER = """Given the following demographic description, is this a realistic, plausible person?

DEMOGRAPHIC: {demographic}

Answer with ONLY one word: "Yes" or "No"
"""

PROMPT_DEMOGRAPHIC_MUTATOR = """You are rewriting a clinical case to match a new patient demographic.

Given:
- Original QUESTION: {original_question}
- NEW DEMOGRAPHIC: {new_demographic}
- DISEASE: {disease}

Task: Rewrite by:
- Updating demographics (age, location, background) to match {new_demographic}
- Keeping disease and symptoms compatible with {disease}
- PRESERVE ORIGINAL STYLE: If first-person ("I", "My"), keep first-person. If third-person, keep third-person.
- Keep language natural and conversational
- Do NOT explicitly name the disease

Return ONLY the rewritten question.
"""

PROMPT_CONSUMER_REWRITE = """Convert this clinical case description into a first-person consumer-style complaint.

CLINICAL TEXT: {clinical_text}

Task:
- Rewrite in patient's voice (I, my, me)
- Use layperson language, avoid medical jargon where possible
- Keep all key information: symptoms, timing, exposures, travel, animals
- Preserve disease consistency (don't add/remove major features)
- 2-4 sentences
- Keep the style natural and conversational

Return ONLY the rewritten text.
"""

PROMPT_CONTEXT_COMBINATION = """You rewrite clinical cases by selectively combining context segments.

Given:
- ATTRIBUTES: {attributes}
- GENERAL_SYMPTOMS: {general_symptoms}
- SPECIFIC_SYMPTOMS: {specific_symptoms}
- LOCATION: {location}
- RISK_FACTORS: {risk_factors}
- INCLUDE_LIST: {include_list}

Task: Construct a single third-person clinical question (2-4 sentences) using ONLY segments in INCLUDE_LIST.
- You may reorder segments and add minimal glue words (and, who, commas)
- Do NOT invent new facts or locations
- Do NOT mention segments not in INCLUDE_LIST
- Do NOT name the disease

Return ONLY the rewritten question text.
"""

PROMPT_FILTER_CHECKER = """Quality check: Is this synthetic case acceptable?

DISEASE: {disease}
CASE DESCRIPTION: {case_text}

Criteria:
- PLAUSIBLE for {disease}?
- Internally consistent (demographics, location, exposures, symptoms make sense)?
- No key symptoms that contradict {disease}?
- No contradictions in text?

Answer with EXACTLY one word: "keep" or "discard"
"""

PROMPT_SYMPTOM_PLAUSIBLE_FILTER = """Consider the following QUESTION and medical condition.

QUESTION: {question}
DISEASE: {disease}

Do the symptoms in the QUESTION reasonably match {disease}?

Answer with ONLY: "Yes" or "No"
"""

In [16]:
class AugmentationPipeline:
    def __init__(self, model: str = "claude-3-5-sonnet-20241022"):
        self.client = anthropic.Anthropic()
        self.model = model

    def call_claude(self, prompt: str, temperature: float = 0.7) -> str:
        """Call Claude API with given prompt"""
        message = self.client.messages.create(
            model=self.model,
            max_tokens=2048,
            temperature=temperature,
            messages=[
                {"role": "user", "content": prompt}
            ]
        )
        return message.content[0].text.strip()

    def call_gemini(self, prompt: str, temperature: float = 0.7) -> str:
        """Call Gemini API using gemini-3-flash-preview"""
        # Select the specific model from your list
        gemini_model = genai.GenerativeModel('gemini-3-flash-preview')

        # Configure generation parameters to match your Claude settings
        generation_config = genai.types.GenerationConfig(
            temperature=temperature,
            max_output_tokens=2048
        )

        try:
            response = gemini_model.generate_content(
                prompt,
                generation_config=generation_config
            )
            return response.text
        except Exception as e:
            print(f"Error calling Gemini: {e}")
            return ""

    def step_1_symptom_semantic_mutator(self, question: str, disease: str, n_variants: int = 3) -> list[dict]:
        """STEP 1: Generate semantic symptom variants formatted as records"""
        print(f"\\n[STEP 1] SYMPTOM_SEMANTIC_MUTATOR: Generating {n_variants} variants...")

        prompt = PROMPT_SEMANTIC_MUTATOR.format(
            question=question,
            disease=disease,
            n_variants=n_variants
        )

        response = self.call_gemini(prompt)
        print(f"Raw response: {response}")

        # 1. Safety Clean: Remove backticks just in case the model ignores the prompt
        cleaned_response = response.replace("```json", "").replace("```", "").strip()
        print(f"Cleansed response: {cleaned_response}")
        try:
            # 2. Parse the JSON
            parsed_data = json.loads(cleaned_response)
            print(f"Parsed data: {parsed_data}")
            # Handle if it returns a dict (like your example: "newquestion1": "...")
            if isinstance(parsed_data, dict):
                raw_questions = list(parsed_data.values())
            elif isinstance(parsed_data, list):
                raw_questions = parsed_data
            else:
                raise ValueError("Output format not recognized (expected list or dict)")

            # 3. Format into the desired List of JSON objects
            # We limit to n_variants and attach the original 'disease' as ground_truth
            formatted_records = [
                {
                    'question_text': q_text,
                    'ground_truth': disease
                }
                for q_text in raw_questions[:n_variants]
            ]

            print(f"✓ Generated {len(formatted_records)} formatted records")
            return formatted_records

        except (json.JSONDecodeError, ValueError) as e:
            print(f"⚠ Error parsing/formatting: {e}")
            print(f"--- RAW RESPONSE ---\n{response}\n--------------------")
            raise e

    def step_2_demographic_sampler(self, disease: str, n_demographics: int = 3) -> list[str]:
        """STEP 2: Generate realistic demographics"""
        print(f"\\n[STEP 2] DEMOGRAPHIC_SAMPLER: Generating {n_demographics} demographics...")

        prompt = PROMPT_DEMOGRAPHIC_SAMPLER.format(
            disease=disease,
            n_demographics=n_demographics
        )

        response = self.call_gemini(prompt)

        # 1. Safety Clean: Remove backticks just in case the model ignores the prompt
        cleaned_response = response.replace("```json", "").replace("```", "").strip()

        try:
            demographics = json.loads(cleaned_response)

            if isinstance(demographics, dict):
                demographics = list(demographics.values())
            elif isinstance(demographics, list):
                demographics = demographics
            else:
                raise ValueError("Output format not recognized (expected list or dict)")
            print(f"✓ Generated {len(demographics)} demographics")
            return demographics[:n_demographics]
        except json.JSONDecodeError as e:
            print(f"⚠ Failed to parse JSON, using raw response")
            raise e

    def step_3_demographic_plausibility_filter(self, demographic: str) -> bool:
        """STEP 3: Validate demographic plausibility"""
        prompt = PROMPT_DEMOGRAPHIC_PLAUSIBLE_FILTER.format(demographic=demographic)
        response = self.call_gemini(prompt, temperature=0.0)
        is_plausible = response.lower().strip() == "yes"
        status = "✓ PLAUSIBLE" if is_plausible else "✗ IMPLAUSIBLE"
        print(f"  {status}: {demographic[:50]}...")
        return is_plausible

    def step_4_demographic_mutator(self, question: str, demographics: list[str], disease: str) -> list[dict]:
        """STEP 4: Merge a LIST of demographics into the case"""
        print(f"\n[STEP 4] DEMOGRAPHIC_MUTATOR: Merging {len(demographics)} demographics into case...")

        generated_records = []

        for i, demographic in enumerate(demographics):
            # 1. Prepare the prompt for this specific demographic
            prompt = PROMPT_DEMOGRAPHIC_MUTATOR.format(
                original_question=question,
                new_demographic=demographic,
                disease=disease
            )

            try:
                # 2. Call Gemini
                response = self.call_gemini(prompt)

                # 3. Clean the response (remove potential Markdown wrapping)
                #    Gemini might wrap plain text in quotes or backticks
                cleaned_response = response.replace("```", "").strip()

                # 4. Create the dictionary record
                record = {
                    'question_text': cleaned_response,
                    'ground_truth': disease
                }

                generated_records.append(record)
                print(f"  ✓ [{i+1}/{len(demographics)}] Generated variant")

            except Exception as e:
                print(f"  ⚠ [{i+1}/{len(demographics)}] Failed: {e}")
                # Optional: Append a None record or skip
                continue

        print(f"✓ Completed. Generated {len(generated_records)} total records.")
        return generated_records

    def step_5_symptom_disease_plausibility_filter(self, question: str, disease: str) -> bool:
        """STEP 5: Check if merged case still plausible for disease"""
        prompt = PROMPT_SYMPTOM_PLAUSIBLE_FILTER.format(
            question=question,
            disease=disease
        )

        response = self.call_gemini(prompt, temperature=0.0)
        is_plausible = response.lower().strip() == "yes"
        status = "✓ VALID" if is_plausible else "✗ INVALID"
        print(f"  {status}: Case-disease match")
        return is_plausible

    def step_6_consumer_rewrite(self, question: str) -> str:
        """STEP 6: Convert to consumer/patient voice"""
        print(f"\\n[STEP 6] CONSUMER_STYLE_REWRITE: Converting to first-person...")

        prompt = PROMPT_CONSUMER_REWRITE.format(clinical_text=question)
        response = self.call_gemini(prompt)
        print(f"✓ Converted to consumer style")
        return response

    def step_7_filter_checker(self, case_text: str, disease: str) -> bool:
        """STEP 7: Final quality gate"""
        print(f"\\n[STEP 7] FILTER_CHECKER: Final quality check...")

        prompt = PROMPT_FILTER_CHECKER.format(
            disease=disease,
            case_text=case_text
        )

        response = self.call_gemini(prompt, temperature=0.0)
        keep = response.lower().strip() == "keep"
        status = "✓ KEEP" if keep else "✗ DISCARD"
        print(f"  {status}")
        return keep


pipeline = AugmentationPipeline()

In [9]:
record = evaluate_data[0]

In [10]:
test_question = record["question_text"]
disease = record["ground_truth"]
print(test_question)
print(disease)

How can I get treated for Guinea worm? I live in Sarh area in Southeast Chari.
Guinea worm


In [11]:
genai.configure(api_key=os.environ["GEMINI_API_KEY"])
models = genai.list_models()
for m in models:
    print(m.name, "→", m.supported_generation_methods)

models/embedding-gecko-001 → ['embedText', 'countTextTokens']
models/gemini-2.5-flash → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.5-pro → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp → ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-001 → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-exp-image-generation → ['generateContent', 'countTokens', 'bidiGenerateContent']
models/gemini-2.0-flash-lite-001 → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-lite → ['generateContent', 'countTokens', 'createCachedContent', 'batchGenerateContent']
models/gemini-2.0-flash-lite-preview-02-05 → ['generateContent', 'countTokens', '

In [17]:
semantic_variants = pipeline.step_1_symptom_semantic_mutator(
    test_question,
    disease,
    n_variants=2
)

\n[STEP 1] SYMPTOM_SEMANTIC_MUTATOR: Generating 2 variants...
Raw response: [
  "I've noticed a very painful swelling on my left ankle that feels like it's burning. Just yesterday, a small blister formed and popped after I waded into the pond near our farm in Kyabé. Now, I can see something thin and white starting to poke out of the open sore. It hurts so much I can barely walk to the fields. Where should I go to get this managed properly?",
  "About a week ago, I started feeling really itchy all over and had a bit of a fever and nausea. Now, a large, angry-looking blister has appeared on my right foot, and it feels like it's on fire. I usually get my drinking water from the local water hole near the border of Salamat. I'm worried because I see a thread-like thing coming out of the wound. What are the steps for getting medical help for this in my region?"
]
Cleansed response: [
  "I've noticed a very painful swelling on my left ankle that feels like it's burning. Just yesterday, a smal

In [13]:
test_question

'How can I get treated for Guinea worm? I live in Sarh area in Southeast Chari.'

In [187]:
semantic_variants

[{'question_text': 'I have a very painful, burning blister on my lower leg that appeared a few days after I returned from the fishing camps near Bongor. It feels like the skin is on fire, and now a small white tip is visible inside the opening of the sore. I usually drink water straight from the river when I am out working. Who can help me get this thing out of my leg safely?',
  'ground_truth': 'Guinea worm'},
 {'question_text': 'For the past week, my foot has been incredibly swollen and I have been feeling quite feverish. A large, itchy bleb has now formed on my heel, and it hurts so much that I can hardly walk to my farm in Moïssala. Many people in my community use the same stagnant pond for drinking water during the dry season. Where is the nearest clinic that handles these types of emerging infections?',
  'ground_truth': 'Guinea worm'}]

In [120]:
demographics = pipeline.step_2_demographic_sampler(disease, n_demographics=2)

\n[STEP 2] DEMOGRAPHIC_SAMPLER: Generating 2 demographics...
✓ Generated 2 demographics


In [121]:
demographics

['A low-income adolescent Dinka male from a rural farming settlement in Aweil West County, South Sudan',
 'A young adult Baguirmi female from a low-income riverside fishing community in the Chari-Baguirmi Region of Chad']

In [169]:
plausible_demo = [d for d in demographics if pipeline.step_3_demographic_plausibility_filter(d)]
plausible_demo

  ✓ PLAUSIBLE: A low-income adolescent Dinka male from a rural fa...
  ✓ PLAUSIBLE: A young adult Baguirmi female from a low-income ri...


['A low-income adolescent Dinka male from a rural farming settlement in Aweil West County, South Sudan',
 'A young adult Baguirmi female from a low-income riverside fishing community in the Chari-Baguirmi Region of Chad']

In [183]:
demographic_variants = pipeline.step_4_demographic_mutator(test_question, plausible_demo, disease)


[STEP 4] DEMOGRAPHIC_MUTATOR: Merging 2 demographics into case...
  ✓ [1/2] Generated variant
  ✓ [2/2] Generated variant
✓ Completed. Generated 2 total records.


In [184]:
demographic_variants

[{'question_text': "I am a young boy from a farming village in Aweil West, and I have a burning blister on my foot with a long white string poking out. It's very painful to walk—how can I get this treated?",
  'ground_truth': 'Guinea worm'},
 {'question_text': "I’m a young woman living in a fishing village along the river here in the Chari-Baguirmi region, and I’ve developed a very painful, burning blister on my foot that now has a thin white string poking out of it. Since my family doesn't have much money, where can I go to get help and have this thing removed safely?",
  'ground_truth': 'Guinea worm'}]

In [206]:
# Create a single list containing all 'question_text' values
variants = [item.get('question_text') for item in demographic_variants + semantic_variants]

In [207]:
variants

["I am a young boy from a farming village in Aweil West, and I have a burning blister on my foot with a long white string poking out. It's very painful to walk—how can I get this treated?",
 "I’m a young woman living in a fishing village along the river here in the Chari-Baguirmi region, and I’ve developed a very painful, burning blister on my foot that now has a thin white string poking out of it. Since my family doesn't have much money, where can I go to get help and have this thing removed safely?",
 'I have a very painful, burning blister on my lower leg that appeared a few days after I returned from the fishing camps near Bongor. It feels like the skin is on fire, and now a small white tip is visible inside the opening of the sore. I usually drink water straight from the river when I am out working. Who can help me get this thing out of my leg safely?',
 'For the past week, my foot has been incredibly swollen and I have been feeling quite feverish. A large, itchy bleb has now form

In [210]:
plausible_questions = [gen_ques for gen_ques in variants if pipeline.step_5_symptom_disease_plausibility_filter(gen_ques, disease)]

  ✓ VALID: Case-disease match
  ✓ VALID: Case-disease match
  ✓ VALID: Case-disease match
  ✓ VALID: Case-disease match


In [230]:
clinical_test_question = evaluate_data[63].get("question_text")

In [231]:
clinical_test_question

'Patient is a 20 year old male who has been having fever on and off for the past week. They appear malnourished, show signs of anemia including fatigue, headache and heart palpitations and symptoms of an enlarged spleen - fullness and pain in the left upper belly. He is of the Turkana tribe and is responsible for herding his family’s cattle. What is the most likely diagnosis for this condition?'

In [232]:
consumer_style_question = pipeline.step_6_consumer_rewrite(clinical_test_question)

\n[STEP 6] CONSUMER_STYLE_REWRITE: Converting to first-person...
✓ Converted to consumer style


In [233]:
consumer_style_question

'I’ve been suffering from a fever that comes and goes for a week while out herding my family’s cattle in Turkana. I feel incredibly weak and thin, my head is throbbing, and my heart feels like it’s racing. I also have this painful, heavy fullness in the upper left part of my belly.'